In [1]:
%load_ext autoreload
%autoreload 2

## Финальный проект по курсу "Рекомендательные системы."

#### Загрузка библиотек и скриптов

In [116]:
import pandas as pd
import numpy as np
from pandas import MultiIndex
import os
os.chdir('C://users//andrei//downloads//lection2//')

from lightgbm import LGBMClassifier
from catboost import CatBoostRanker
from sklearn.metrics import classification_report, precision_recall_curve

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares  
from implicit.nearest_neighbours import bm25_weight, tfidf_weight, ItemItemRecommender, CosineRecommender

from lightgbm import LGBMClassifier

from src.utils import prefilter_items
from src.recommenders import MainRecommender
from src.metrics import recall_at_k, precision_at_k

#### Загрузка данных

In [117]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')
data_test = pd.read_csv('retail_test.csv')

Приведем названия столбцов к общему виду

In [118]:
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

#### Разделение на обучение и валидацию

Модель первого этапа (предожение рекомендаций методами библиотеки Implicit) обучим на всём датасете, кроме 6 последних недель, валидировать её будем на оставшейся части;

Модель второго этапа (отбор рекомендаций) будем обучать на 6 последних неделях.



In [253]:
# Размер валидационной выборки в неделях.
val_size = 6

# Данные для обучения модели 1-го уровня.
data_train = data[data['week_no'] < data['week_no'].max() - val_size]

# Валидационные данные для модели 1-го уровня.
data_val = data[(data['week_no'] >= data['week_no'].max() - val_size)]

# Тренировочные для модели 2-го уровня
# data_train_2 = data_val.copy()

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


#### Префильтрация товаров

Отберём товары согласно нашей функции prefilter из модуля utils.py.
А именно:

    - Убираем самые популярные товары;
    - Самые непопулярные товары;
    - Товары которые не продавались в течении 12 месяцев;
    - Не интересные для рекомендаций (субъективный выбор некоторых категорий, судя по топ-проданных товаров данные собраны о магазинах на АЗС, поэтому нет смысла оставлять раздел в котором продаётся бензин.)
    - Слишком дешевые товары. Дорогие товары было решено не трогать. Вряд ли они попадут в ТОП и будут рекомендованы.
    - В обработанный датасет не включаем "фейковый" товар.

In [254]:
old = data_train['item_id'].nunique()
data_train = prefilter_items(data_train, item_features, take_n_popular=6000)
new = data_train['item_id'].nunique()
print(f'Количество товаров уменьшено c - {old} до {new}.')

Количество товаров уменьшено c - 85334 до 6000.


In [198]:
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,1.39
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,1.50


In [255]:
# Уберем из валидации тех пользователей которых нет в тренировочной части
old = data_val['user_id'].nunique()
users = data_train['user_id'].unique()
data_val = data_val.loc[data_val['user_id'].isin(users)]
# data_train_2 = data_train_2.loc[data_train_2['user_id'].isin(users)]
new = data_val['user_id'].nunique()
print(f'Количество пользователей уменьшено с {old} до {new}')

Количество пользователей уменьшено с 2197 до 2185


### Этап 1 отбор рекомендаций, выбор модели отбора

In [256]:
# Инициализируем наш модуль
recommender = MainRecommender(data_train, item_features)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.012000799179077148 seconds
  warnings.warn(


  0%|          | 0/6000 [00:00<?, ?it/s]

In [257]:
# Подготовим валидационный датасет для оценки качества
result = data_val.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[829323, 835108, 836423, 851515, 875240, 87737..."
1,2,"[895388, 8357614, 12301772, 821083, 828106, 83..."


In [258]:
def make_recommendations(data, name_model, N=50):
    rec_name = name_model[0]
    rec_model = name_model[1]
    data[rec_name] = data['user_id'].apply(lambda x: rec_model(x, N=N))

In [259]:
def calc_recall(data, top_k):
    for col_name in data.columns[2:]:
        yield col_name, data.apply(lambda row: recall_at_k(row[col_name], row['actual'], k=top_k), axis=1).mean()

In [260]:
def calc_precision(data, top_k):
    for col_name in data.columns[2:]:
        yield col_name, data.apply(lambda row: precision_at_k(row[col_name], row['actual'], k=top_k), axis=1).mean()

In [261]:
own_rec = ('own_recs', recommender.get_own_recommendations) # K=2
als_rec = ('als_recs', recommender.get_als_recommendations)
cosine_rec = ('cosine_recs', recommender.get_cosine_recommender)

In [262]:
for rec in (own_rec, als_rec, cosine_rec):
    make_recommendations(result, rec)

In [263]:
result.head(2)

,user_id,actual,own_recs,als_recs,cosine_recs
0,1,"[829323, 835108, 836423, 851515, 875240, 87737...","[961554, 1004906, 9527290, 940947, 1006184, 85...","[856942, 8090541, 5577022, 940947, 1082269, 10...","[856942, 9297615, 8090541, 5577022, 940947, 96..."
1,2,"[895388, 8357614, 12301772, 821083, 828106, 83...","[961554, 1004906, 5569230, 916122, 1106523, 11...","[5569230, 8090521, 916122, 1133018, 8090537, 1...","[5569230, 8090521, 8090537, 916122, 1106523, 1..."


In [264]:
sorted(calc_precision(result, 5), key=lambda x: x[1], reverse=True)

[('cosine_recs', 0.30627002288329475),
 ('als_recs', 0.28320366132723),
 ('own_recs', 0.23716247139587854)]

Методами библиотеки Implicit лучшие результаты показывает алгоритм косинусной близости. 
Но так как мы будем использовать 2-х уровневую модель посмотрим на результаты по оценке recall_at_k

In [265]:
sorted(calc_recall(result, 50), key=lambda x: x[1], reverse=True)

[('own_recs', 0.11558733627498949),
 ('cosine_recs', 0.1068878863729549),
 ('als_recs', 0.10359746998162882)]

Здесь лучшим оказался метод на основе прошлых покупок пользователя (Был применен k=2, т.о. нельзя сказать, что мы рекомендовали абсолютно тоже, что пользователь уже покупал.)


### Этап 2 обучение модели 2-го уровня для отбора кандидатов.

Напишем функцию для формирования датасета для модели второго уровня. На вход будет приниматься валидационный и тренировочный датасеты, можно указавать параметр target(для обучения или для теста)

In [273]:
def level_2_data(user_data, data_train, N=30, target=True, item_features=item_features, user_features=user_features):
            
    def pref_department(user):
        a1 = np.zeros(len(dl))
        for el in cat_count.loc[user, ('department')].items():
            for item in el[1]:
                a1[dl.index(item)] += 1
        ind = np.argpartition(a1, -5)[-5:]
        
        return [dl[el] for el in ind[np.argsort(a1[ind])][::-1]]
    
    # Формируем список пользователй
    data_train_2 = pd.DataFrame(user_data['user_id'].unique())
    data_train_2.columns = ['user_id']
    # Фильтруем
    train_users = data_train['user_id'].unique()
    data_train_2 = data_train_2[(data_train_2['user_id'].isin(train_users))]
    data_train_2.reset_index(drop=True, inplace=True)
    # Формируем рекомендации
    data_train_2['recs'] = data_train_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N))
    # Превращаем датасет с рекомендациями в датасет с данными для второй модели
    s = data_train_2.apply(lambda x: pd.Series(x['recs']), axis=1).stack().reset_index(level=1, drop=True)
    s.name = 'item_id'
    data_train_2 = data_train_2.drop('recs', axis=1).join(s)
    
    # Если создаем обучающий датасет
    if target:
        # Сначала помечаем все рекомендации 1
        data_train_2['flag'] = 1
        # копируем в обучающий датасет для модели 2-го уровня колонки user_id и item_id и 1-го валидационного датасета.
        user_lvl_2 = user_data[['user_id', 'item_id']].copy()
        # удаляем дубликаты, те товары которые покупались неоднократно
        user_lvl_2 = user_lvl_2.drop_duplicates()
        user_lvl_2['target'] = 1  # тут только покупки 
        # объединяем датасет с рекомендациями и реальными покупками, те товары, которые присутствуют в обоих, получают целевой признак.
        data_train_2 = data_train_2.merge(user_lvl_2, on=['user_id', 'item_id'], how='left')
        # Остальные поля признака заполняем 0.
        data_train_2['target'].fillna(0, inplace=True)
        data_train_2.drop('flag', axis=1, inplace=True)
    
    # Объединяем с датасетами признаки товаров и признаки пользователй
    data_train_2 = data_train_2.merge(item_features, on='item_id', how='left')
    data_train_2 = data_train_2.merge(user_features, on='user_id', how='left')
    
#     p = data_train.loc[:, ('sales_value')] / np.maximum(data_train.loc[:, ('quantity')], 1)
#     p.name = 'price'
#     data_train = data_train.join(p)
    data_train = data_train.merge(item_features, left_on = 'item_id', right_on = 'item_id', how='left')
    
    """Добавляем новые признаки"""
    # Признак: цена товара
    price = data_train.groupby('item_id')['price'].median()
    # data_train_2['price'] = data_train_2['item_id'].apply(lambda x: price.loc[x])
    price.name = 'price'
    data_train_2 = data_train_2.merge(price, how='left', on='item_id')
    
    
    # Признак: сумма среднего чека

    check = data_train.groupby('user_id')['sales_value'].sum() /  data_train.groupby('user_id')['basket_id'].nunique()
    check.name = 'mean_check'
    data_train_2.merge(check, how='left', on='user_id')
    
    # Признак: количество покупок в неделю
    weeks_quanty = round(data_train['day'].max() / 7, ndigits=-1)
    checks_per_week = data_train.groupby('user_id')['basket_id'].nunique() / weeks_quanty
    checks_per_week.name = 'checks_per_week'
    data_train_2 = data_train_2.merge(checks_per_week, how='left', on='user_id')
    
    def calc_week_count(user, department, q=weeks_quanty):
        if department in cat_count.loc[user].index:
            return cat_count.loc[user, department][0] / q
        else:
            return 0
    
    # Количество покупок товаров одной категории в чеке на каждого пользователя
    cat_count = data_train.groupby(['user_id', 'department']).agg({'item_id': 'count'})
    data_train_2['cat_per_week'] = data_train_2.apply(lambda x: calc_week_count(x.user_id, x.department), axis=1)
    
    # Признак: среднее количество покупок категорий товаров на пользователя в неделю. Возможно есть смысл посчитать на чек.
    q_user = data_train['user_id'].nunique() # общее количество пользователей
    # группировка купленных товаров по отделам
    mean_cat_per_week = (data_train.groupby('department')['item_id'].count() / q_user) / weeks_quanty 
    data_train_2['mean_cat_per_week'] = data_train_2['department'].apply(lambda x: mean_cat_per_week[x])
    
    # Признак: отношение покупок отдельным клиентом категорий товаров к среднему количеству покупок всеми пользователями
    # в неделю.
    data_train_2['user_vs_mean'] = data_train_2['cat_per_week'] / data_train_2['mean_cat_per_week']
    
    # кол-во магазинов, в которых продавался товар
    store = data_train.groupby(['item_id'])['store_id'].nunique().reset_index()
    store.columns = ['item_id', 'n_stores']
    store.name = 'n_stores'
    data_train_2 = data_train_2.merge(store, how='left', on='item_id')
    
    # Признак: средняя цена товара по каждой категории.
    mean_price = data_train.groupby('department').agg({'price': 'mean'})
    data_train_2['mean_price'] = data_train_2['department'].apply(lambda x: mean_price.loc[x][0])
        
    return data_train_2

In [328]:
%%time
data_train_2 = level_2_data(data_val, data_train, N=30)
data_train_2.head(2)

CPU times: total: 1min 22s
Wall time: 1min 23s


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,hh_comp_desc,household_size_desc,kid_category_desc,price,checks_per_week,cat_per_week,mean_cat_per_week,user_vs_mean,n_stores,mean_price
0,84,1004906,0.0,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,NaN,...,NaN,NaN,NaN,2.69,0.333333,0.033333,0.60013,0.055544,111,2.303313
1,84,961554,0.0,69,PRODUCE,Private,CARROTS,CARROTS MINI PEELED,1 LB,NaN,...,NaN,NaN,NaN,1.69,0.333333,0.033333,0.60013,0.055544,112,2.303313


In [329]:
# Доля целевого признака.
data_train_2['target'].mean()

0.18108314263920672

In [330]:
# Выделим целевой признак
# X_train = data_train_2.drop(columns=['target', 'kid_category_desc', 'department', 'homeowner_desc', 'brand', 'predict'], axis=1)
X_train = data_train_2.drop(columns=['target', 'kid_category_desc', 'department', 'homeowner_desc', 'brand'], axis=1)
y_train = data_train_2['target']

In [331]:
# Укажем категориальные переменные.
cat_feats = X_train.columns[2:11].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')
cat_feats

['manufacturer',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'hh_comp_desc',
 'household_size_desc']

In [277]:
X_train.head(2)

,user_id,item_id,manufacturer,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,hh_comp_desc,household_size_desc,price,checks_per_week,cat_per_week,mean_cat_per_week,user_vs_mean,store_id,mean_price
0,84,1004906,69,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,NaN,NaN,NaN,NaN,NaN,2.69,0.333333,0.033333,0.60013,0.055544,111,2.303313
1,84,961554,69,CARROTS,CARROTS MINI PEELED,1 LB,NaN,NaN,NaN,NaN,NaN,1.69,0.333333,0.033333,0.60013,0.055544,112,2.303313


In [332]:
# Обучим модель.
lgb = LGBMClassifier(objective='binary', max_depth=12, n_estimators=700, learning_rate=0.15, reg_alpha=0.0, categorical_column=cat_feats)
lgb.fit(X_train, y_train)
# class_weight='balanced', min_child_samples=5 
train_preds = lgb.predict(X_train)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


In [333]:
# Посмотрим на метрики.
print(classification_report(y_train, train_preds))

              precision    recall  f1-score   support

         0.0       0.91      0.99      0.95     53680
         1.0       0.96      0.56      0.71     11870

    accuracy                           0.92     65550
   macro avg       0.93      0.78      0.83     65550
weighted avg       0.92      0.92      0.91     65550



In [293]:
# data_train_2.drop('predict', axis=1, inplace = True)
# Получим вероятности классов
target_pred = pd.Series(lgb.predict_proba(X_train)[:, 1])
# Добавим эти данные в наш датасет, чтобы получить рекомендации
target_pred.name = 'predict'
data_train_2 = data_train_2.join(target_pred)

Функция для формирования итоговых предсказаний на основе лучшей вероятности целевого признака.

In [281]:
def get_lgb_recommendation(user, data):
    recs, index = [], 0
    # получаем pd.Series индексы + значения предсказаний.
    predict = data.loc[(data['user_id'] == user, 'predict')]
    # сортируем значения получаем топ индексов предсказаний
    predict_index = predict.sort_values(ascending=False).index.tolist()[:20]
    if predict_index:

        return data.iloc[predict_index[:5]].item_id.tolist()
    else:
        recs = recommender.get_als_recommendations(user, 5)
        return recs

In [294]:
result['lgb'] = result['user_id'].apply(lambda x: get_lgb_recommendation(x, data_train_2))
result.apply(lambda x: precision_at_k(x['lgb'], x['actual'], k=5), axis=1).mean()

0.5904805491990835

#### Получение предсказаний для тестовой выборки

In [283]:
'''Уберем из тестовых данных тех пользователей которых нет в обученной модели.
Для них можно:
- сформировать рекомендации на основе модуля MainRecommender если они есть в обучающем датасете;
- предложить топ-популярных товаров если это новые пользователи.'''
old = data_test['user_id'].nunique()
users = data_train_2['user_id'].unique()
data_test = data_test.loc[data_test['user_id'].isin(users)]
new = data_test['user_id'].nunique()
print(f'Количество пользователей уменьшено с {old} до {new}')

Количество пользователей уменьшено с 1781 до 1781


In [284]:
# Сформируем данные для 2ой валидации.
result_2 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_2.columns = ['user_id', 'actual']

result_2.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [334]:
%%time
data_test_2 = level_2_data(data_test, data_train, N=30, target=False)
data_test_2.head(2)

CPU times: total: 1min 8s
Wall time: 1min 10s


,user_id,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,...,hh_comp_desc,household_size_desc,kid_category_desc,price,checks_per_week,cat_per_week,mean_cat_per_week,user_vs_mean,n_stores,mean_price
0,1340,961554,69,PRODUCE,Private,CARROTS,CARROTS MINI PEELED,1 LB,NaN,NaN,...,NaN,NaN,NaN,1.69,0.244444,0.077778,0.600130,0.129602,112,2.303313
1,1340,916122,4314,MEAT,National,CHICKEN,CHICKEN BREAST BONELESS,,NaN,NaN,...,NaN,NaN,NaN,3.76,0.244444,0.077778,0.264905,0.293606,112,4.528548


In [236]:
X_train.columns

Index(['user_id', 'item_id', 'manufacturer', 'commodity_desc',
       'sub_commodity_desc', 'curr_size_of_product', 'age_desc',
       'marital_status_code', 'income_desc', 'hh_comp_desc',
       'household_size_desc', 'price', 'checks_per_week', 'cat_per_week',
       'mean_cat_per_week', 'user_vs_mean', 'mean_price'],
      dtype='object')

In [335]:
X_test = data_test_2.drop(columns=['kid_category_desc', 'department', 'homeowner_desc', 'brand'], axis=1)
X_test[cat_feats] = X_test[cat_feats].astype('category')

,manufacturer,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,hh_comp_desc,household_size_desc
0,69,CARROTS,CARROTS MINI PEELED,1 LB,NaN,NaN,NaN,NaN,NaN
1,4314,CHICKEN,CHICKEN BREAST BONELESS,,NaN,NaN,NaN,NaN,NaN
2,2852,BEEF,PRIMAL,,NaN,NaN,NaN,NaN,NaN
3,2854,BEEF,PRIMAL,,NaN,NaN,NaN,NaN,NaN
4,2,GRAPES,GRAPES WHITE,18 LB,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
53425,3726,DELI MEATS,MEAT:HAM BULK,,NaN,NaN,NaN,NaN,NaN
53426,69,TOMATOES,TOMATOES VINE RIPE PKG,4 CT,NaN,NaN,NaN,NaN,NaN
53427,2,SALAD BAR,SALAD BAR FRESH FRUIT,,NaN,NaN,NaN,NaN,NaN
53428,612,CANNED JUICES,CRANBERRY JUICE (50% AND UNDER,64 OZ,NaN,NaN,NaN,NaN,NaN


In [336]:
# Получим вероятности классов
target_pred = pd.Series(lgb.predict_proba(X_test)[:, 1])
# Добавим эти данные в наш датасет, чтобы получить рекомендации
target_pred.name = 'predict'
data_test_2 = data_test_2.join(target_pred)

In [337]:
result_2['lgb'] = result_2['user_id'].apply(lambda x: get_lgb_recommendation(x, data_test_2))
result_2.apply(lambda x: precision_at_k(x['lgb'], x['actual'], k=5), axis=1).mean()

0.26895002807411317

#### Для сравнения получим предсказания используя весь обучающий датасет.

In [338]:
old = data['item_id'].nunique()
data_for_test = prefilter_items(data, item_features, take_n_popular=6000)
new = data_for_test['item_id'].nunique()
print(f'Количество товаров уменьшено c - {old} до {new}.')

Количество товаров уменьшено c - 89051 до 6000.


In [339]:
recommender = MainRecommender(data_for_test, item_features)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.01600027084350586 seconds
  warnings.warn(


  0%|          | 0/6000 [00:00<?, ?it/s]

In [340]:
for rec in (own_rec, als_rec, cosine_rec):
    make_recommendations(result_2, rec)
result_2.head(2)

,user_id,actual,lgb,own_recs,als_recs,cosine_recs
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[9655212, 5577022, 10149640, 1041796, 856942]","[961554, 1004906, 9527290, 940947, 1006184, 85...","[856942, 8090541, 5577022, 940947, 1082269, 10...","[856942, 9297615, 8090541, 5577022, 940947, 96..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[1106523, 1053690, 899624, 940947, 1133018]","[961554, 1004906, 5569230, 916122, 1106523, 11...","[5569230, 8090521, 916122, 1133018, 8090537, 1...","[5569230, 8090521, 8090537, 916122, 1106523, 1..."


In [341]:
sorted(calc_precision(result_2, 5), key=lambda x: x[1], reverse=True)

[('lgb', 0.26895002807411317),
 ('cosine_recs', 0.21763054463784162),
 ('als_recs', 0.20280741156653334),
 ('own_recs', 0.16720943290286203)]

In [344]:
path = 'C://users//andrei//ds//rs//'
df = result_2[['user_id', 'lgb']].copy()
df.to_csv(path + 'predictions_BAM_RS.csv', index=False)
df.head(2)

,user_id,lgb
0,1,"[9655212, 5577022, 10149640, 1041796, 856942]"
1,2,"[1106523, 1053690, 899624, 940947, 1133018]"
